# Implementing self-editing memory from scratch

There are a variety of ways to enable long-term memory in LLM agents, such as RAG and recursive summarization. The **MemGPT** paper first introduced the notion of *self-editing memory*. Essentially, offload memory management to the LLM. After all, the LLM is the most "intelligent" part of our programs, so why not have the LLM figure out memory instead of hard coding some solution?

In this notebook, we'll walk through how to use OpenAI's tool calling to implement some simple memory management tools.

## Preparation

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
   <p> 💻 &nbsp; <b>Create/Add to</b> <code>requirements.txt</code>
    
<code>
    python_dotenv==1.0.1
    letta==0.6.50
</code>
</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI models can vary with each execution due to their dynamic, probabilistic nature. Your results may differ from those shown in the video.</p>

## Section 0: Setup OpenAI

In [19]:
import os
from dotenv import load_dotenv, find_dotenv
                                                                                                                                    
def load_env():
    _ = load_dotenv(find_dotenv())

def get_openai_api_key():
    load_env()
    openai_api_key = os.getenv("OPENAI_API_KEY")
    return openai_api_key
openai_api_key = get_openai_api_key()

In [20]:
from openai import OpenAI
import os

client = OpenAI(
    api_key=openai_api_key
)

## Section 1: Breaking down the LLM context window
LLM's have a *context window* (the set of tokens that go into the model) that is limited in size. That means we need to be smart about what we place into the context. Usually the context window for an agent is strucuted in a certain way. Depending on the agent framework, the structure will vary, but usually the context window contains: 
* A *system prompt* instructing the agent's behavior
* A *conservation history* of previous conversations

Because context windows are limited, only some of the conversation history can be included. Some frameworks will also place a recursive summary in the context, or retrieve relevant messages from an external database and also place them into the context. In MemGPT, we also reserver additional sections of the context for: 
* A *recursive summary* of all previous conversations
* A *core memory* section that is read-writeable by the agent

### A simple agent's context window
In the code below, you can see how we can define an agent with a system prompt. The system prompt will be included in every chat completions request as the first message, while later messages will change over time as the user and assistant exchange messages. 

In [21]:
model = "gpt-4o-mini"

In [22]:
system_prompt = "You are a chatbot."

In [23]:
# Make the completion request with the tool usage
chat_completion = client.chat.completions.create(
    model=model,
    messages=[
        # system prompt: always included in the context window 
        {"role": "system", "content": system_prompt}, 
        # chat history (evolves over time)
        {"role": "user", "content": "What is my name?"}, 
    ]
)
chat_completion.choices[0].message.content

'I don’t know your name unless you tell me. What would you like me to call you?'

### Adding memory to the context 
Similar to how we always start a chat completitions request with a system prompt, we can also prefix the list of messages with a prompt containing important memories. Lets see how we can add a memory section to the context window, and have the agent use that memory to respond to user messages.  

In [24]:
agent_memory = {"human": "Name: Bob"}
system_prompt = "You are a chatbot. " \
+ "You have a section of your context called [MEMORY] " \
+ "that contains information relevant to your conversation"

In [25]:
import json


chat_completion = client.chat.completions.create(
    model=model,
    messages=[
        # system prompt 
        {"role": "system", "content": system_prompt + "[MEMORY]\n" + json.dumps(agent_memory)},
        # chat history 
        {"role": "user", "content": "What is my name?"},
    ],
)
chat_completion.choices[0].message.content

'Your name is Bob.'

In the next section, we'll show you can to make this memory read-writeable by the agent. 

## Section 2: Modifing the memory with tools 

We first need to define a memory save tool. In order to allow the agent to save new memory, we implement a simple tool that appends to a section of the memory dictionary which we will pass to the agent.  

### Defining a memory editing tool 
Instead of directly providing the name in the agent's memory, we'll instead start with a blank memory object and provide a function that can edit the memory object. 

In [26]:
agent_memory = {"human": "", "agent": ""}

def core_memory_save(section: str, memory: str): 
    agent_memory[section] += '\n' 
    agent_memory[section] += memory 

To inform our agent of this tool and how to use it, we need to create a tool schema that OpenAI can process. This includes a description of how to use the tool, and the parameters the agent must generate to input into the tool. 

In [27]:

# tool description 
core_memory_save_description = "Save important information about you," \
+ "the agent or the human you are chatting with."

# arguments into the tool (generated by the LLM)
# defines what the agent must generate to input into the tool 
core_memory_save_properties = \
{
    # arg 1: section of memory to edit
    "section": {
        "type": "string",
        "enum": ["human", "agent"],
        "description": "Must be either 'human' " \
        + "(to save information about the human) or 'agent'" \
        + "(to save information about yourself)",            
    },
    # arg 2: memory to save
    "memory": {
        "type": "string",
        "description": "Memory to save in the section",
    },
}

# tool schema (passed to OpenAI)
core_memory_save_metadata = \
    {
        "type": "function",
        "function": {
            "name": "core_memory_save",
            "description": core_memory_save_description,
            "parameters": {
                "type": "object",
                "properties": core_memory_save_properties,
                "required": ["section", "memory"],
            },
        }
    }

Now, we can pass the tool call into the agent! 

In [28]:


agent_memory = {"human": ""}
system_prompt = "You are a chatbot. " \
+ "You have a section of your context called [MEMORY] " \
+ "that contains information relevant to your conversation"

chat_completion = client.chat.completions.create(
    model=model,
    messages=[
        # system prompt 
        {"role": "system", "content": system_prompt}, 
        # memory 
        {"role": "system", "content": "[MEMORY]\n" + json.dumps(agent_memory)},
        # chat history 
        {"role": "user", "content": "My name is Bob"},
    ],
    # tool schemas 
    tools=[core_memory_save_metadata]
)
response = chat_completion.choices[0]
response

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_p2LRco7zwPs1dIg2FFncGTNB', function=Function(arguments='{"section":"human","memory":"The user\'s name is Bob."}', name='core_memory_save'), type='function')]))

### Executing the tool 
Unfortunately, OpenAI isn't going to actually execute the tool, so we have to do this ourselves. Lets take the arguments specified in the tool call response we just got to run the tool. 

In [29]:
arguments = json.loads(response.message.tool_calls[0].function.arguments)
arguments

{'section': 'human', 'memory': "The user's name is Bob."}

In [30]:
# run the function with the specified arguments 
core_memory_save(**arguments)

Now, we can view the memory object that has been updated: 

In [31]:
agent_memory

{'human': "\nThe user's name is Bob."}

### Running the next agent step 
Now, we can see how the agent responds differently as the memory has been updated. 

In [ ]:
chat_completion = client.chat.completions.create(
    model=model,
    messages=[
        # system prompt 
        {"role": "system", "content": system_prompt}, 
        # memory 
        {"role": "system", "content": "[MEMORY]\n" + json.dumps(agent_memory)},
        # chat history 
        {"role": "user", "content": "what is my name"},
    ],
    tools=[core_memory_save_metadata]
)
response = chat_completion.choices[0]
response.message

Congrats! You've now implemented the a memory editing function. Can you think of how you could add support for other data structures?

## Implementing an agentic loop
In our current implementation, the agent can only take one step at a time: it can either edit memory, or respond to the user. However, ideally we want our agent to support *multi-step reasoning*, so it can combine multiple actions together. For example, if we tell the agent our name is "Bob", we want the agent to both edit its memory and also return back a message to us. 

We can implement multi-step reasoning by calling the `client.chat.completions.create(...)` in a loop, and allowing the agent to choose whether to continue its reasoning steps or to break out of the loop. For simplicity, we will assume that an agent response that is *not* a tool call breaks out of the reasoning loop yields control back to the user.


In [ ]:
agent_memory = {"human": ""}

In [ ]:
system_prompt_os = system_prompt \
+ "\n. You must either call a tool (core_memory_save) or" \
+ "write a response to the user. " \
+ "Do not take the same actions multiple times!" \
+ "When you learn new information, make sure to always" \
+ "call the core_memory_save tool." 

We implement a simple step function for the agent. The function responds to the user message but allows the agent to take multiple actions in sequence, and returns to the user when a message (that does not call a tool) is sent. 

In [ ]:
def agent_step(user_message, chat_history = []): 

    # prefix messages with system prompt and memory
    messages = [
        # system prompt 
        {"role": "system", "content": system_prompt_os}, 
        # memory
        {
            "role": "system", 
            "content": "[MEMORY]\n" + json.dumps(agent_memory)
        },
    ] 

    # append the chat history 
    messages += chat_history
    

    # append the most recent message
    messages.append({"role": "user", "content": user_message})
    
    # agentic loop 
    while True: 
        chat_completion = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=[core_memory_save_metadata]
        )
        response = chat_completion.choices[0]

        # update the messages with the agent's response 
        messages.append(response.message)

        # if NOT calling a tool (responding to the user), return 
        if not response.message.tool_calls: 
            messages.append({
                "role": "assistant", 
                "content": response.message.content
            })
            return response.message.content

        # if calling a tool, execute the tool
        if response.message.tool_calls: 
            print("TOOL CALL:", response.message.tool_calls[0].function)

            # add the tool call response to the message history 
            messages.append({"role": "tool", "tool_call_id": response.message.tool_calls[0].id, "name": "core_memory_save", "content": f"Updated memory: {json.dumps(agent_memory)}"})

            # parse the arguments from the LLM function call
            arguments = json.loads(response.message.tool_calls[0].function.arguments)

            # run the function with the specified arguments
            core_memory_save(**arguments)

Now, we can observe the agent take multiple actions when we sent it a message: 

In [ ]:
agent_step("my name is bob.")

Now, the agent is able to both edit its memory *and* generate a response to the user that uses the updated memory in a single step. 

Although in this example, we only support a single tools and responding to the user, this same structure can be used to implement more complex reasoning loops that combine many tools. In MemGPT, all actions (even a response to the user) is a tool, where some tools (such as sending a message) break the agent reasoning loop, while other tools (such as searching the archival memory store or editing memory) do not. 

You've now implemented an agent with self-editing memory and multi-step reasoning :)